In [2]:
import time
import statsapi
import pandas as pd
import numpy as np

### Load in MLB schedule history from statsapi

#### 1. Download all Missing/Incomplete Seasons to .json files

In [3]:
from src.data.schedule import Downloader
from pathlib import Path
base_dir = str(Path(Path.cwd()).resolve().parent)
#Downloader(2010, f'{base_dir}/data/raw/schedule/').download_games()

### 2. Load in data from files and construct dataframe

* Exclude games that were suspended and completed later; only include games that were completed on the day they were scheduled
    </br>Handle in 2 stages:
    1. for last_n stats, treat game as having occurred on the date finished
    2. remove said games from final dataset (try with and without)

In [4]:
from src.data.schedule import DfMaker
from pathlib import Path
base_dir = str(Path(Path.cwd()).resolve().parent)
data_dir = base_dir + '/data'
"""
schedule_df = DfMaker(data_dir).make_df()
schedule_df.head()
"""

'\nschedule_df = DfMaker(data_dir).make_df()\nschedule_df.head()\n'

In [5]:
"""
status_options = [
    'Final',
    'Completed Early: Rain',
    'Completed Early',
    'Completed Early: Wet Grounds'
]
schedule_df = schedule_df[schedule_df.status.isin(status_options)]
# Keep list of ids with duplicates (2)
duplicate_game_ids = schedule_df[schedule_df.game_id.duplicated()].game_id.unique()
# keep the second game of duplicate game_ids (1)
schedule_df = schedule_df.drop_duplicates(subset='game_id', keep='last')
schedule_df.game_id.value_counts().value_counts()
schedule_df.reset_index(drop=True).to_feather(f'{data_dir}/interim/schedule.feather')
"""

"\nstatus_options = [\n    'Final',\n    'Completed Early: Rain',\n    'Completed Early',\n    'Completed Early: Wet Grounds'\n]\nschedule_df = schedule_df[schedule_df.status.isin(status_options)]\n# Keep list of ids with duplicates (2)\nduplicate_game_ids = schedule_df[schedule_df.game_id.duplicated()].game_id.unique()\n# keep the second game of duplicate game_ids (1)\nschedule_df = schedule_df.drop_duplicates(subset='game_id', keep='last')\nschedule_df.game_id.value_counts().value_counts()\nschedule_df.reset_index(drop=True).to_feather(f'{data_dir}/interim/schedule.feather')\n"

In [6]:
import pandas as pd
schedule_df = pd.read_feather(f'{data_dir}/interim/schedule.feather')
schedule_df

,game_id,game_datetime,game_date,game_type,status,away_name,home_name,away_id,home_id,doubleheader,...,national_broadcasts,series_status,summary,winning_team,losing_team,winning_pitcher,losing_pitcher,save_pitcher,losing_Team,season
0,276989,2010-03-02 17:05:00+00:00,2010-03-02,E,Completed Early,"State College of Florida, Manatee-Sarasota",Pittsburgh Pirates,230,134,N,...,[],PIT wins Exhibition,"2010-03-02 - State College of Florida, Manatee...",None,None,None,None,None,None,2010
1,276990,2010-03-02 18:05:00+00:00,2010-03-02,E,Final,Florida Southern Mocs,Detroit Tigers,228,116,N,...,[],DET wins Exhibition,2010-03-02 - Florida Southern Mocs (1) @ Detro...,Detroit Tigers,Florida Southern Mocs,Phil Coke,Gus Schlosser,None,None,2010
2,276991,2010-03-02 18:10:00+00:00,2010-03-02,S,Final,Atlanta Braves,New York Mets,144,121,N,...,[MLBN],NYM leads 1-0,2010-03-02 - Atlanta Braves (2) @ New York Met...,New York Mets,Atlanta Braves,Bobby Livingston,Jesse Chavez,Ryota Igarashi,None,2010
3,276992,2010-03-03 18:05:00+00:00,2010-03-03,S,Final,Tampa Bay Rays,Baltimore Orioles,139,110,N,...,[],BAL leads 1-0,2010-03-03 - Tampa Bay Rays (2) @ Baltimore Or...,Baltimore Orioles,Tampa Bay Rays,Will Ohman,Matt Garza,None,None,2010
4,276993,2010-03-03 18:05:00+00:00,2010-03-03,S,Final,Detroit Tigers,Toronto Blue Jays,116,141,N,...,[],DET leads 1-0,2010-03-03 - Detroit Tigers (7) @ Toronto Blue...,Detroit Tigers,Toronto Blue Jays,Daniel Schlereth,Willie Collazo,Robbie Weinhardt,None,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38420,748537,2023-10-25 00:07:00+00:00,2023-10-24,L,Final,Arizona Diamondbacks,Philadelphia Phillies,109,143,N,...,"[TBS-INT, TBS]",AZ wins 4-3,2023-10-24 - Arizona Diamondbacks (4) @ Philad...,Arizona Diamondbacks,Philadelphia Phillies,Ryan Thompson,Ranger Suarez,Paul Sewald,None,2023
38421,748542,2023-10-28 00:05:00+00:00,2023-10-27,W,Final,Arizona Diamondbacks,Texas Rangers,109,140,N,...,"[FOX-INT, FOX]",TEX leads 1-0,2023-10-27 - Arizona Diamondbacks (5) @ Texas ...,Texas Rangers,Arizona Diamondbacks,Jose Leclerc,Miguel Castro,None,None,2023
38422,748541,2023-10-29 00:03:00+00:00,2023-10-28,W,Final,Arizona Diamondbacks,Texas Rangers,109,140,N,...,"[FOX-INT, FOX]",Series tied 1-1,2023-10-28 - Arizona Diamondbacks (9) @ Texas ...,Arizona Diamondbacks,Texas Rangers,Merrill Kelly,Jordan Montgomery,None,None,2023
38423,748540,2023-10-31 00:03:00+00:00,2023-10-30,W,Final,Texas Rangers,Arizona Diamondbacks,140,109,N,...,"[FOX-INT, FOX]",TEX leads 2-1,2023-10-30 - Texas Rangers (3) @ Arizona Diamo...,Texas Rangers,Arizona Diamondbacks,Jon Gray,Brandon Pfaadt,Jose Leclerc,None,2023


In [ ]:
schedule_df

### 3. Construct Dataframe Containing the matchups with last 5 and last 10 game states for home and away teams

##### Construct from Season class

In [6]:
from src.features import Season
"""
years = schedule_df.season.unique()
last_ns = [5, 10]
matchups_df = pd.concat(tuple([Season(schedule_df, year, include_today=True).get_matchups_df(last_ns) for year in years]))
matchups_df.sort_values('game_datetime').tail(10)
"""

"\nyears = schedule_df.season.unique()\nlast_ns = [5, 10]\nmatchups_df = pd.concat(tuple([Season(schedule_df, year, include_today=True).get_matchups_df(last_ns) for year in years]))\nmatchups_df.sort_values('game_datetime').tail(10)\n"

##### Save to feather

In [7]:
#matchups_df.reset_index(drop=True).to_feather(f'{data_dir}/interim/matchups.feather')

#### Load from file

In [8]:
matchups_df = pd.read_feather(f'{data_dir}/interim/matchups.feather')

#### Download Game Logs

In [9]:

from src.data.game_log import Downloader
"""
seasons = np.sort(schedule_df.season.unique())
game_ids = {year:list(schedule_df[(schedule_df.game_type=='R')&(schedule_df.season==year)].game_id.unique()) for year in seasons}

dl = Downloader(game_ids, f'{base_dir}/data/raw/game_logs/')
dl.download_seasons()
"""

"\nseasons = np.sort(schedule_df.season.unique())\ngame_ids = {year:list(schedule_df[(schedule_df.game_type=='R')&(schedule_df.season==year)].game_id.unique()) for year in seasons}\n\ndl = Downloader(game_ids, f'{base_dir}/data/raw/game_logs/')\ndl.download_seasons()\n"

##### Collate Game Logs into Games Stats Data Frame

In [10]:
load_dir = f'{base_dir}/data/raw/game_logs/'
"""
batter_df = pd.concat(tuple([pd.read_feather(f'{load_dir}/{season}/batters_{game}.feather') for season in seasons for game in game_ids[season]])).reset_index(drop=True)
pitcher_df = pd.concat(tuple([pd.read_feather(f'{load_dir}/{season}/pitchers_{game}.feather') for season in seasons for game in game_ids[season]])).reset_index(drop=True)



game_ids = {year:list(schedule_df[(schedule_df.game_type=='R')&(schedule_df.season==year)].game_id.unique()) for year in seasons}
load_dir = f'{base_dir}/data/raw/game_logs/'
batter_df = pd.concat(tuple([pd.read_feather(f'{load_dir}/{season}/batters_{game}.feather') for season in seasons for game in game_ids[season]])).reset_index(drop=True)
pitcher_df = pd.concat(tuple([pd.read_feather(f'{load_dir}/{season}/pitchers_{game}.feather') for season in seasons for game in game_ids[season]])).reset_index(drop=True)
#batter_df.to_feather(f'{data_dir}/interim/batter_df.feather')
#pitcher_df.to_feather(f'{data_dir}/interim/pitcher_df.feather')
"""

"\nbatter_df = pd.concat(tuple([pd.read_feather(f'{load_dir}/{season}/batters_{game}.feather') for season in seasons for game in game_ids[season]])).reset_index(drop=True)\npitcher_df = pd.concat(tuple([pd.read_feather(f'{load_dir}/{season}/pitchers_{game}.feather') for season in seasons for game in game_ids[season]])).reset_index(drop=True)\n"

In [11]:
#batter_df.to_feather(f'{data_dir}/interim/batter_df.feather')
#pitcher_df.to_feather(f'{data_dir}/interim/pitcher_df.feather')

#### Load pitcher_df

In [12]:
pitcher_df = pd.read_feather(f'{data_dir}/interim/pitcher_df.feather')

In [13]:
pitcher_df.head()

,GS,namefield,ip,h,r,er,bb,k,hr,p,s,era,name,personId,note,team_id,team_abbrev,team_name,game_id
0,True,Sabathia,5.1,6,5,5,2,4,0,104,58,8.44,Sabathia,282332,,147,NYY,Yankees,263816
1,False,"Robertson (BS, 1)",0.2,1,0,0,0,0,0,6,6,0.00,Robertson,502085,"(BS, 1)",147,NYY,Yankees,263816
2,False,"Park (L, 0-1)(BS, 1)",0.2,3,3,2,0,1,1,22,14,27.00,Park,120221,"(L, 0-1)(BS, 1)",147,NYY,Yankees,263816
3,False,Marte,0.0,0,0,0,1,0,0,5,1,-.--,Marte,150122,,147,NYY,Yankees,263816
4,False,Chamberlain,1.1,2,1,1,1,0,0,33,22,6.75,Chamberlain,501955,,147,NYY,Yankees,263816


### Convert Pitcher Df to dataframe of season cumulative and recent trend stats for starting pitchers for the following statistics:
1. IP/start (game)
2. ERA
3. WHIP
4. ER
5. Hits/IP
6. Walks/IP (bb)
7. Strikeouts/9 (k)
8. Homeruns/IP (r)
9. pitches/start
10. strike fractions (s/p)


In [14]:
from src.features import Starters, StarterSeason

In [23]:
numbers_of_days = [16, 31]
starter_stats_df = Starters(pitcher_df, schedule_df).aggregate_stats(numbers_of_days)

In [ ]:
from src.features import Starters, StarterSeason
numbers_of_days = [16, 31]
starter_stats_df = Starters(pitcher_df, schedule_df).aggregate_stats(numbers_of_days)
starter_stats_df.dropna(subset=['GS_cum', 'ip_cum', 'er_cum', 'h_cum', 'bb_cum', 'k_cum', 'hr_cum',
       'p_cum', 's_cum', 'ip_ave_cum', 'p_start_ave_cum', 's_start_ave_cum',
       'strike_ratio_cum', 'k9_cum', 'bb9_cum', 'hr9_cum', 'whip_cum',
       'era_cum', 'GS_16_days', 'ip_16_days', 'er_16_days', 'h_16_days',
       'bb_16_days', 'k_16_days', 'hr_16_days', 'p_16_days', 's_16_days',
       'ip_ave_16_days', 'p_start_ave_16_days', 's_start_ave_16_days',
       'strike_ratio_16_days', 'k9_16_days', 'bb9_16_days', 'hr9_16_days',
       'whip_16_days', 'era_16_days', 'GS_31_days', 'ip_31_days', 'er_31_days',
       'h_31_days', 'bb_31_days', 'k_31_days', 'hr_31_days', 'p_31_days',
       's_31_days', 'ip_ave_31_days', 'p_start_ave_31_days',
       's_start_ave_31_days', 'strike_ratio_31_days', 'k9_31_days',
       'bb9_31_days', 'hr9_31_days', 'whip_31_days', 'era_31_days']).reset_index(drop=True).to_feather(f'{data_dir}/interim/starter_stats.feather')

In [28]:
starter_stats_df.shape

(64966, 76)

In [27]:
name = 'Hughes'
starter_stats_df[starter_stats_df['name'].str.contains(name)].head(50)

,GS,namefield,ip,h,r,er,bb,k,hr,p,...,s_31_days,ip_ave_31_days,p_start_ave_31_days,s_start_ave_31_days,strike_ratio_31_days,k9_31_days,bb9_31_days,hr9_31_days,whip_31_days,era_31_days
0,True,"Hughes, P (W, 1-0)",5.0,3,2,2,5,6,1,108,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,"Hughes, P (W, 2-0)",7.1,1,1,1,2,10,0,101,...,66.0,5.000000,108.000000,66.000000,0.611111,10.800000,9.000000,1.800000,1.600000,3.600000
2,True,"Hughes, P",5.2,2,1,1,4,2,0,109,...,136.0,6.050000,104.500000,68.000000,0.650718,11.900826,5.206612,0.743802,0.909091,2.231405
3,True,"Hughes, P (W, 3-0)",7.0,4,0,0,1,6,0,99,...,204.0,5.766667,106.000000,68.000000,0.641509,9.364162,5.722543,0.520231,0.982659,2.080925
4,True,"Hughes, P (W, 4-0)",7.0,7,2,2,1,7,0,101,...,273.0,6.075000,104.250000,68.250000,0.654676,8.888889,4.444444,0.370370,0.905350,1.481481
5,True,"Hughes, P (W, 5-0)",7.0,5,0,0,1,8,0,101,...,343.0,6.260000,103.600000,68.600000,0.662162,8.913738,3.738019,0.287540,0.958466,1.725240
6,True,"Hughes, P",5.0,6,5,5,1,3,2,104,...,348.0,6.660000,102.200000,69.600000,0.681018,8.918919,2.432432,0.000000,0.840841,1.081081
7,True,"Hughes, P (L, 5-1)",5.2,8,4,4,3,7,0,117,...,419.0,6.383333,102.500000,69.833333,0.681301,8.459530,2.349869,0.469974,0.913838,2.114883
8,True,"Hughes, P (W, 6-1)",7.0,5,2,2,1,8,1,109,...,437.0,6.066667,105.166667,72.833333,0.692552,8.159341,2.719780,0.494505,1.181319,2.967033
9,True,"Hughes, P (W, 7-1)",7.0,6,1,1,1,7,0,101,...,376.0,6.240000,106.400000,75.200000,0.706767,9.519231,2.019231,0.865385,1.217949,3.750000


In [ ]:

starter_stats_df.dropna(subset=['GS_cum', 'ip_cum', 'er_cum', 'h_cum', 'bb_cum', 'k_cum', 'hr_cum',
       'p_cum', 's_cum', 'ip_ave_cum', 'p_start_ave_cum', 's_start_ave_cum',
       'strike_ratio_cum', 'k9_cum', 'bb9_cum', 'hr9_cum', 'whip_cum',
       'era_cum', 'GS_16_days', 'ip_16_days', 'er_16_days', 'h_16_days',
       'bb_16_days', 'k_16_days', 'hr_16_days', 'p_16_days', 's_16_days',
       'ip_ave_16_days', 'p_start_ave_16_days', 's_start_ave_16_days',
       'strike_ratio_16_days', 'k9_16_days', 'bb9_16_days', 'hr9_16_days',
       'whip_16_days', 'era_16_days', 'GS_31_days', 'ip_31_days', 'er_31_days',
       'h_31_days', 'bb_31_days', 'k_31_days', 'hr_31_days', 'p_31_days',
       's_31_days', 'ip_ave_31_days', 'p_start_ave_31_days',
       's_start_ave_31_days', 'strike_ratio_31_days', 'k9_31_days',
       'bb9_31_days', 'hr9_31_days', 'whip_31_days', 'era_31_days']).reset_index(drop=True).to_feather(f'{data_dir}/interim/starter_stats.feather')

### Test framework

In [15]:
pitcher_id = 282332
season = 2010
starter_df = pitcher_df[pitcher_df.GS].merge(schedule_df[['game_id', 'game_datetime', 'season']]).reset_index(drop=True)
ss = StarterSeason(starter_df, season,  pitcher_id, [16, 31])
df1 = ss.get_season_df()

In [31]:
df1.columns

Index(['GS', 'namefield', 'ip', 'h', 'r', 'er', 'bb', 'k', 'hr', 'p', 's',
       'era', 'name', 'personId', 'note', 'team_id', 'team_abbrev',
       'team_name', 'game_id', 'game_datetime', 'season', 'days_rest',
       'GS_cum', 'ip_cum', 'er_cum', 'h_cum', 'bb_cum', 'k_cum', 'hr_cum',
       'p_cum', 's_cum', 'ip_ave_cum', 'p_start_ave_cum', 's_start_ave_cum',
       'strike_ratio_cum', 'k9_cum', 'bb9_cum', 'hr9_cum', 'whip_cum',
       'era_cum', 'GS_16_days', 'ip_16_days', 'er_16_days', 'h_16_days',
       'bb_16_days', 'k_16_days', 'hr_16_days', 'p_16_days', 's_16_days',
       'ip_ave_16_days', 'p_start_ave_16_days', 's_start_ave_16_days',
       'strike_ratio_16_days', 'k9_16_days', 'bb9_16_days', 'hr9_16_days',
       'whip_16_days', 'era_16_days', 'GS_31_days', 'ip_31_days', 'er_31_days',
       'h_31_days', 'bb_31_days', 'k_31_days', 'hr_31_days', 'p_31_days',
       's_31_days', 'ip_ave_31_days', 'p_start_ave_31_days',
       's_start_ave_31_days', 'strike_ratio_31_days', '

In [33]:
df1[['game_datetime', 'game_id', 'ip','er']]

,game_datetime,game_id,ip,er
0,2010-04-05 00:05:00+00:00,263816,5.1,5
1,2010-04-10 19:10:00+00:00,263877,7.2,0
2,2010-04-16 23:05:00+00:00,263960,6.0,1
3,2010-04-22 19:35:00+00:00,264037,8.0,3
4,2010-04-28 23:05:00+00:00,264119,7.2,3
5,2010-05-03 23:05:00+00:00,264185,8.0,1
6,2010-05-08 19:10:00+00:00,264258,4.2,3
7,2010-05-13 17:05:00+00:00,264320,6.0,6
8,2010-05-18 23:05:00+00:00,264389,7.0,1
9,2010-05-24 00:05:00+00:00,264470,5.0,5


In [35]:
response = statsapi.get('game_winProbability', {'gamePk': 266164})

In [48]:
ind = 0
print(response[ind]['homeTeamWinProbability'], response[ind]['playEndTime'], response[ind]['atBatIndex'], response[ind]['result'])

ind = 10
print(response[ind]['homeTeamWinProbability'], response[ind]['playEndTime'], response[ind]['atBatIndex'], response[ind]['result'])

46.400000000000006 2010-09-28T23:10:16.000Z 0 {'type': 'atBat', 'event': 'Single', 'eventType': 'single', 'description': 'Derek Jeter singles on a ground ball to center fielder Vernon Wells.', 'rbi': 0, 'awayScore': 0, 'homeScore': 0, 'isOut': False}
42.699999999999996 2010-09-28T23:28:17.000Z 10 {'type': 'atBat', 'event': 'Groundout', 'eventType': 'field_out', 'description': 'Lance Berkman grounds out, second baseman Aaron Hill to first baseman Lyle Overbay.', 'rbi': 0, 'awayScore': 1, 'homeScore': 0, 'isOut': True}


In [38]:
response[0].keys()

dict_keys(['result', 'about', 'count', 'matchup', 'pitchIndex', 'actionIndex', 'runnerIndex', 'runners', 'playEvents', 'credits', 'flags', 'homeTeamWinProbability', 'awayTeamWinProbability', 'homeTeamWinProbabilityAdded', 'contextMetrics', 'playEndTime', 'atBatIndex'])

In [29]:
statsapi.ENDPOINTS

{'attendance': {'url': 'https://statsapi.mlb.com/api/{ver}/attendance',
  'path_params': {'ver': {'type': 'str',
    'default': 'v1',
    'leading_slash': False,
    'trailing_slash': False,
    'required': True}},
  'query_params': ['teamId',
   'leagueId',
   'season',
   'date',
   'leagueListId',
   'gameType',
   'fields'],
  'required_params': [['teamId'], ['leagueId'], ['leagueListid']]},
 'awards': {'url': 'https://statsapi.mlb.com/api/{ver}/awards{awardId}{recipients}',
  'path_params': {'ver': {'type': 'str',
    'default': 'v1',
    'leading_slash': False,
    'trailing_slash': False,
    'required': True},
   'awardId': {'type': 'str',
    'default': None,
    'leading_slash': True,
    'trailing_slash': False,
    'required': False},
   'recipients': {'type': 'bool',
    'default': True,
    'True': '/recipients',
    'False': '',
    'leading_slash': False,
    'trailing_slash': False,
    'required': False}},
  'query_params': ['sportId', 'leagueId', 'season', 'hydrate',